# Importing Dependences

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import math
import joblib
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Dataset Preparation

In [ ]:
#!gdown 1sVFwyVpYNURhIgAGTq8qfkl8Z3t1fBu_ #data fix
!gdown 1PpEkZ4a2MVYAYGzNio8aoDTAQRfi1JQs #data fix baru

Downloading...
From: https://drive.google.com/uc?id=1PpEkZ4a2MVYAYGzNio8aoDTAQRfi1JQs
To: /content/data_modelling_rev2.xlsx
100% 107k/107k [00:00<00:00, 73.8MB/s]


In [ ]:
#df = pd.read_excel("/content/data_modelling_rev1.xlsx")
df = pd.read_excel("/content/data_modelling_rev2.xlsx")

df

,State ID,State,County,City,Longitude,Latitude,Zip Code,Density_2020,Land Area (Sq. Kilometer)_2020,Density_2010,...,Total CS,Total CS Private,Total CS Public,Total ESVE (Estimated),Total ESVE (Estimated) Private,Total ESVE (Estimated) Public,Total Non EV,Total Population,Per capita income,Median Value (Dollars)
0,WA,Washington,King,Auburn,47.30919,-122.26426,98001,713.9,48.263062,715.414905,...,6,1,5,14,2,12,11116,34455,35620,361400
1,WA,Washington,King,Auburn,47.30820,-122.21567,98002,1829.6,18.554329,1833.659823,...,5,1,4,12,2,9,4014,33947,28275,270300
2,WA,Washington,King,Federal Way,47.30596,-122.31465,98003,1659.9,29.787939,1643.775383,...,7,0,7,17,0,17,14513,49445,30059,326900
3,WA,Washington,King,Bellevue,47.61865,-122.20548,98004,1979.1,18.829266,2035.377909,...,103,2,101,251,4,246,37981,37265,99774,1445500
4,WA,Washington,King,Bellevue,47.61494,-122.16814,98005,1126.7,19.005947,1102.692130,...,49,1,48,119,2,117,16366,21414,75452,963600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,46.04821,-118.89575,99363,2.0,78.500000,1.361529,...,0,0,0,0,0,0,309,157,13719,137500
594,WA,Washington,Adams,Washtucna,46.79144,-118.30614,99371,0.6,545.000000,0.619576,...,0,0,0,0,0,0,309,327,24669,80000
595,WA,Washington,Asotin,Anatone,46.09041,-117.27336,99401,0.8,470.000000,0.660622,...,0,0,0,0,0,0,0,376,24091,268000
596,WA,Washington,Asotin,Asotin,46.18680,-117.14922,99402,1.8,887.222222,2.152604,...,0,0,0,0,0,0,0,1597,34638,243300


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   State ID                        598 non-null    object 
 1   State                           598 non-null    object 
 2   County                          598 non-null    object 
 3   City                            598 non-null    object 
 4   Longitude                       598 non-null    float64
 5   Latitude                        598 non-null    float64
 6   Zip Code                        598 non-null    int64  
 7   Density_2020                    598 non-null    float64
 8   Land Area (Sq. Kilometer)_2020  598 non-null    float64
 9   Density_2010                    598 non-null    float64
 10  Land Area (Sq. Kilometer)_2010  598 non-null    float64
 11  Total EV                        598 non-null    int64  
 12  Total CS                        598 

# Dataset Preprocessing

In [ ]:
df = df.rename(columns = {"State ID": "state_id",
                          "State": "state",
                          "County": "county",
                          "City": "city",
                          "Longitude": "long",
                          "Latitude": "lat",
                          "Zip Code": "zip_code",
                          "Total EV": "total_ev",
                          "Total ESVE (Estimated)": "total_cs",
                          "Total Non EV ": "total_non_ev",
                          "Total Population": "total_population",
                          "Per capita income": "annual_income",
                          "Median Value (Dollars)": "house_valuation",
                          "Land Area (Sq. Kilometer)_2010": "area_land"})

df["density_pop"] = df["total_population"] / df["area_land"]
df["density_cs"] = df["total_cs"] / df["area_land"]
df["density_ev"] = df["total_ev"] / df["area_land"]
df["density_non_ev"] = df["total_non_ev"] / df["area_land"]
df["house_valuation"] = df["house_valuation"].fillna(0)

df

,state_id,state,county,city,long,lat,zip_code,Density_2020,Land Area (Sq. Kilometer)_2020,Density_2010,...,Total ESVE (Estimated) Private,Total ESVE (Estimated) Public,total_non_ev,total_population,annual_income,house_valuation,density_pop,density_cs,density_ev,density_non_ev
0,WA,Washington,King,Auburn,47.30919,-122.26426,98001,713.9,48.263062,715.414905,...,2,12,11116,34455,35620,361400,715.414905,0.290692,9.260631,230.809812
1,WA,Washington,King,Auburn,47.30820,-122.21567,98002,1829.6,18.554329,1833.659823,...,2,9,4014,33947,28275,270300,1833.659823,0.648184,8.642459,216.817702
2,WA,Washington,King,Federal Way,47.30596,-122.31465,98003,1659.9,29.787939,1643.775383,...,0,17,14513,49445,30059,326900,1643.775383,0.565157,9.906868,482.477746
3,WA,Washington,King,Bellevue,47.61865,-122.20548,98004,1979.1,18.829266,2035.377909,...,4,246,37981,37265,99774,1445500,2035.377909,13.709375,106.670953,2074.485130
4,WA,Washington,King,Bellevue,47.61494,-122.16814,98005,1126.7,19.005947,1102.692130,...,2,117,16366,21414,75452,963600,1102.692130,6.127784,41.710125,842.750509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,46.04821,-118.89575,99363,2.0,78.500000,1.361529,...,0,0,309,157,13719,137500,1.361529,0.000000,0.000000,2.679697
594,WA,Washington,Adams,Washtucna,46.79144,-118.30614,99371,0.6,545.000000,0.619576,...,0,0,309,327,24669,80000,0.619576,0.000000,0.000000,0.585471
595,WA,Washington,Asotin,Anatone,46.09041,-117.27336,99401,0.8,470.000000,0.660622,...,0,0,0,376,24091,268000,0.660622,0.000000,0.000000,0.000000
596,WA,Washington,Asotin,Asotin,46.18680,-117.14922,99402,1.8,887.222222,2.152604,...,0,0,0,1597,34638,243300,2.152604,0.000000,0.010783,0.000000


In [ ]:
df[["total_population",
    "area_land",
    "density_pop",
    "annual_income",
    "house_valuation",
    "total_cs",
    "density_cs",
    "total_non_ev",
    "density_non_ev",
    "total_ev",
    "density_ev"]].corr()

df = df[["state_id", 
         "state", 
         "county", 
         "city", 
         "zip_code",
         "long", 
         "lat", 
         "total_population", 
         "area_land",
         "annual_income", 
         "house_valuation",
         "total_cs",
         "total_ev", 
         "total_non_ev", 
         "density_pop", 
         "density_cs",
         "density_ev",
         "density_non_ev"]]

df

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,house_valuation,total_cs,total_ev,total_non_ev,density_pop,density_cs,density_ev,density_non_ev
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,361400,14,446,11116,715.414905,0.290692,9.260631,230.809812
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,270300,12,160,4014,1833.659823,0.648184,8.642459,216.817702
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,326900,17,298,14513,1643.775383,0.565157,9.906868,482.477746
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,1445500,251,1953,37981,2035.377909,13.709375,106.670953,2074.485130
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,963600,119,810,16366,1102.692130,6.127784,41.710125,842.750509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,137500,0,0,309,1.361529,0.000000,0.000000,2.679697
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,80000,0,0,309,0.619576,0.000000,0.000000,0.585471
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,268000,0,0,0,0.660622,0.000000,0.000000,0.000000
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,243300,0,8,0,2.152604,0.000000,0.010783,0.000000


## Grouping EV

In [ ]:
th = 0.95 # threshold quantile electric vehicle

df["cluster_ev"] = np.nan
df_temp = df.copy()
temp_ev = df_temp["density_ev"].loc[(df_temp["density_ev"].notnull()) &
                                    (df_temp["density_ev"] > 0)].quantile(th)
print(temp_ev)                                                
df_temp["density_ev"].loc[df_temp["density_ev"] > temp_ev] = temp_ev
kmeans_ev = KMeans(n_clusters = 9, random_state = 42).fit(df_temp[["density_ev"]].loc[(df_temp["density_ev"].notnull())].values)
df_temp["cluster_ev"].loc[df_temp["density_ev"].notnull()] = kmeans_ev.predict(df_temp[["density_ev"]].loc[df_temp["density_ev"].notnull()].values)

dict_class = {}
for j in range(9):
    dict_class[j] = max(df_temp["density_ev"].loc[df_temp["cluster_ev"] == j])
dict_class = dict(sorted(dict_class.items(), key=lambda item: item[1]))

df["cluster_ev"].loc[(df["density_ev"].notnull())] = kmeans_ev.predict(df[["density_ev"]].loc[(df["density_ev"].notnull())].values)
df["cluster_ev"] = df[["cluster_ev",
                       "density_ev"]].apply(lambda x: 10 if x["density_ev"] == 0 or math.isnan(x["density_ev"]) else
                                            (9 if x["cluster_ev"] == list(dict_class.keys())[0] else 
                                             (8 if x["cluster_ev"] == list(dict_class.keys())[1] else 
                                              (7 if x["cluster_ev"] == list(dict_class.keys())[2] else 
                                               (6 if x["cluster_ev"] == list(dict_class.keys())[3] else
                                                (5 if x["cluster_ev"] == list(dict_class.keys())[4] else
                                                 (4 if x["cluster_ev"] == list(dict_class.keys())[5] else 
                                                  (3 if x["cluster_ev"] == list(dict_class.keys())[6] else
                                                   (2 if x["cluster_ev"] == list(dict_class.keys())[7] else
                                                    (1 if x["cluster_ev"] == list(dict_class.keys())[8] else x))))))))), axis = 1)

joblib.dump(kmeans_ev, "cluster_ev_demand_internet.gz")
joblib.dump(dict_class, "dict_cluster_ev_demand_internet.gz")

df

64.41496759863863


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,house_valuation,total_cs,total_ev,total_non_ev,density_pop,density_cs,density_ev,density_non_ev,cluster_ev
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,361400,14,446,11116,715.414905,0.290692,9.260631,230.809812,7
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,270300,12,160,4014,1833.659823,0.648184,8.642459,216.817702,7
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,326900,17,298,14513,1643.775383,0.565157,9.906868,482.477746,7
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,1445500,251,1953,37981,2035.377909,13.709375,106.670953,2074.485130,1
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,963600,119,810,16366,1102.692130,6.127784,41.710125,842.750509,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,137500,0,0,309,1.361529,0.000000,0.000000,2.679697,10
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,80000,0,0,309,0.619576,0.000000,0.000000,0.585471,10
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,268000,0,0,0,0.660622,0.000000,0.000000,0.000000,10
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,243300,0,8,0,2.152604,0.000000,0.010783,0.000000,9


In [ ]:
for i in range(1, 11):
    print(i, df["density_ev"].loc[(df["cluster_ev"] == i)].min(), df["density_ev"].loc[(df["cluster_ev"] == i)].max())

1 59.19116626882573 457.01854809777456
2 48.94735063272518 55.63771427515382
3 40.76017493335421 46.256038804093
4 32.099064529095706 37.02065804506816
5 20.406342644849804 28.91582036174212
6 12.59065048095438 19.73024343918161
7 6.229733253030374 12.152758839798528
8 1.8363687921990521 6.090805850871513
9 0.0007052011792404529 1.827701192147803
10 0.0 0.0


## Grouping CS

In [ ]:
th = 0.95 # threshold quantile charging station

df["cluster_cs"] = np.nan
df_temp = df.copy()
temp_cs = df_temp["density_cs"].loc[(df_temp["density_cs"].notnull()) &
                                    (df_temp["density_cs"] > 0)].quantile(th)
print(temp_cs)                                                
df_temp["density_cs"].loc[df_temp["density_cs"] > temp_cs] = temp_cs
kmeans_cs = KMeans(n_clusters = 9, random_state = 42).fit(df_temp[["density_cs"]].loc[(df_temp["density_cs"].notnull())].values)
df_temp["cluster_cs"].loc[df_temp["density_cs"].notnull()] = kmeans_cs.predict(df_temp[["density_cs"]].loc[df_temp["density_cs"].notnull()].values)

dict_class = {}
for j in range(9):
    dict_class[j] = max(df_temp["density_cs"].loc[df_temp["cluster_cs"] == j])
dict_class = dict(sorted(dict_class.items(), key=lambda item: item[1]))

df["cluster_cs"].loc[(df["density_cs"].notnull())] = kmeans_cs.predict(df[["density_cs"]].loc[(df["density_cs"].notnull())].values)
df["cluster_cs"] = df[["cluster_cs",
                       "density_cs"]].apply(lambda x: 10 if x["density_cs"] == 0 or math.isnan(x["density_cs"]) else
                                            (9 if x["cluster_cs"] == list(dict_class.keys())[0] else 
                                             (8 if x["cluster_cs"] == list(dict_class.keys())[1] else 
                                              (7 if x["cluster_cs"] == list(dict_class.keys())[2] else 
                                               (6 if x["cluster_cs"] == list(dict_class.keys())[3] else
                                                (5 if x["cluster_cs"] == list(dict_class.keys())[4] else
                                                 (4 if x["cluster_cs"] == list(dict_class.keys())[5] else 
                                                  (3 if x["cluster_cs"] == list(dict_class.keys())[6] else
                                                   (2 if x["cluster_cs"] == list(dict_class.keys())[7] else
                                                    (1 if x["cluster_cs"] == list(dict_class.keys())[8] else x))))))))), axis = 1)

joblib.dump(kmeans_cs, "cluster_cs_demand_internet.gz")
joblib.dump(dict_class, "dict_cluster_cs_demand_internet.gz")

df

5.555006255997446


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,house_valuation,total_cs,total_ev,total_non_ev,density_pop,density_cs,density_ev,density_non_ev,cluster_ev,cluster_cs
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,361400,14,446,11116,715.414905,0.290692,9.260631,230.809812,7,8
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,270300,12,160,4014,1833.659823,0.648184,8.642459,216.817702,7,7
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,326900,17,298,14513,1643.775383,0.565157,9.906868,482.477746,7,7
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,1445500,251,1953,37981,2035.377909,13.709375,106.670953,2074.485130,1,1
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,963600,119,810,16366,1102.692130,6.127784,41.710125,842.750509,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,137500,0,0,309,1.361529,0.000000,0.000000,2.679697,10,10
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,80000,0,0,309,0.619576,0.000000,0.000000,0.585471,10,10
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,268000,0,0,0,0.660622,0.000000,0.000000,0.000000,10,10
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,243300,0,8,0,2.152604,0.000000,0.010783,0.000000,9,10


In [ ]:
for i in range(1, 11):
    print(i, df["density_cs"].loc[(df["cluster_cs"] == i)].min(), df["density_cs"].loc[(df["cluster_cs"] == i)].max())

1 5.492854205210053 1640.9329765751993
2 4.594564804429822 4.9620270302097085
3 3.4774738576428064 3.988389930715736
4 2.578788045692772 3.170240129638394
5 1.5018923994422213 2.027400256951106
6 0.8238304626768636 1.3113600863117727
7 0.43250330259058467 0.7544736235295034
8 0.11602365957151103 0.37729825098490877
9 0.0009837516831250166 0.10868455364988693
10 0.0 0.0


## Grouping Non EV

In [ ]:
th = 0.95 # threshold quantile non electric vehicle

df["cluster_non_ev"] = np.nan
df_temp = df.copy()
temp_non_ev = df_temp["density_non_ev"].loc[(df_temp["density_non_ev"].notnull()) &
                                            (df_temp["density_non_ev"] > 0)].quantile(th)
print(temp_non_ev)
df_temp["density_non_ev"].loc[df_temp["density_non_ev"] > temp_non_ev] = temp_non_ev
kmeans_non_ev = KMeans(n_clusters = 9, random_state = 42).fit(df_temp[["density_non_ev"]].loc[(df_temp["density_non_ev"].notnull())].values)
df_temp["cluster_non_ev"].loc[df_temp["density_non_ev"].notnull()] = kmeans_non_ev.predict(df_temp[["density_non_ev"]].loc[df_temp["density_non_ev"].notnull()].values)

dict_class = {}
for j in range(9):
    dict_class[j] = max(df_temp["density_non_ev"].loc[df_temp["cluster_non_ev"] == j])
dict_class = dict(sorted(dict_class.items(), key=lambda item: item[1]))

df["cluster_non_ev"].loc[(df["density_non_ev"].notnull())] = kmeans_non_ev.predict(df[["density_non_ev"]].loc[(df["density_non_ev"].notnull())].values)
df["cluster_non_ev"] = df[["cluster_non_ev",
                           "density_non_ev"]].apply(lambda x: 10 if x["density_non_ev"] == 0 or math.isnan(x["density_non_ev"]) else
                                                    (9 if x["cluster_non_ev"] == list(dict_class.keys())[0] else
                                                     (8 if x["cluster_non_ev"] == list(dict_class.keys())[1] else
                                                      (7 if x["cluster_non_ev"] == list(dict_class.keys())[2] else
                                                       (6 if x["cluster_non_ev"] == list(dict_class.keys())[3] else
                                                        (5 if x["cluster_non_ev"] == list(dict_class.keys())[4] else
                                                         (4 if x["cluster_non_ev"] == list(dict_class.keys())[5] else
                                                          (3 if x["cluster_non_ev"] == list(dict_class.keys())[6] else
                                                           (2 if x["cluster_non_ev"] == list(dict_class.keys())[7] else
                                                            (1 if x["cluster_non_ev"] == list(dict_class.keys())[8] else x))))))))), axis = 1)

joblib.dump(kmeans_non_ev, "cluster_non_ev_demand_internet.gz")
joblib.dump(dict_class, "dict_cluster_non_ev_demand_internet.gz")

df

1470.701991399881


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,...,total_cs,total_ev,total_non_ev,density_pop,density_cs,density_ev,density_non_ev,cluster_ev,cluster_cs,cluster_non_ev
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,...,14,446,11116,715.414905,0.290692,9.260631,230.809812,7,8,7
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,...,12,160,4014,1833.659823,0.648184,8.642459,216.817702,7,7,7
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,...,17,298,14513,1643.775383,0.565157,9.906868,482.477746,7,7,5
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,...,251,1953,37981,2035.377909,13.709375,106.670953,2074.485130,1,1,1
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,...,119,810,16366,1102.692130,6.127784,41.710125,842.750509,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,...,0,0,309,1.361529,0.000000,0.000000,2.679697,10,10,9
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,...,0,0,309,0.619576,0.000000,0.000000,0.585471,10,10,9
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,...,0,0,0,0.660622,0.000000,0.000000,0.000000,10,10,10
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,...,0,8,0,2.152604,0.000000,0.010783,0.000000,9,10,10


In [ ]:
for i in range(1, 11):
    print(i, df["density_non_ev"].loc[(df["cluster_non_ev"] == i)].min(), df["density_non_ev"].loc[(df["cluster_non_ev"] == i)].max())

1 1303.0059240757316 22168.02612612194
2 1008.2183893216836 1244.6624881404828
3 798.2993607591815 985.1250329895341
4 563.9195978031456 752.890020496515
5 404.59610304207416 547.2728437795322
6 284.2823329414478 399.5138821687807
7 163.5247020502498 262.9201459793579
8 54.846539900955925 148.5746948243313
9 0.16805742699971127 51.845007744880135
10 0.0 0.0


## Grouping Population

In [ ]:
th = 0.95 # threshold quantile population

df["cluster_pop"] = np.nan
df_temp = df.copy()
temp_pop = df_temp["density_pop"].loc[(df_temp["density_pop"].notnull()) &
                                      (df_temp["density_pop"] > 0)].quantile(th)
print(temp_pop)
df_temp["density_pop"].loc[df_temp["density_pop"] > temp_pop] = temp_pop
kmeans_pop = KMeans(n_clusters = 9, random_state = 42).fit(df_temp[["density_pop"]].loc[(df_temp["density_pop"].notnull())].values)
df_temp["cluster_pop"].loc[df_temp["density_pop"].notnull()] = kmeans_pop.predict(df_temp[["density_pop"]].loc[df_temp["density_pop"].notnull()].values)

dict_class = {}
for j in range(9):
    dict_class[j] = max(df_temp["density_pop"].loc[df_temp["cluster_pop"] == j])
dict_class = dict(sorted(dict_class.items(), key=lambda item: item[1]))

df["cluster_pop"].loc[(df["density_pop"].notnull())] = kmeans_pop.predict(df[["density_pop"]].loc[(df["density_pop"].notnull())].values)
df["cluster_pop"] = df[["cluster_pop",
                        "density_pop"]].apply(lambda x: 10 if x["density_pop"] == 0 or math.isnan(x["density_pop"]) else
                                              (9 if x["cluster_pop"] == list(dict_class.keys())[0] else
                                               (8 if x["cluster_pop"] == list(dict_class.keys())[1] else
                                                (7 if x["cluster_pop"] == list(dict_class.keys())[2] else
                                                 (6 if x["cluster_pop"] == list(dict_class.keys())[3] else
                                                  (5 if x["cluster_pop"] == list(dict_class.keys())[4] else
                                                   (4 if x["cluster_pop"] == list(dict_class.keys())[5] else
                                                    (3 if x["cluster_pop"] == list(dict_class.keys())[6] else
                                                     (2 if x["cluster_pop"] == list(dict_class.keys())[7] else
                                                      (1 if x["cluster_pop"] == list(dict_class.keys())[8] else x))))))))), axis = 1)

joblib.dump(kmeans_pop, "cluster_pop_demand_internet.gz")
joblib.dump(dict_class, "dict_cluster_pop_demand_internet.gz")

df

2383.0742283119325


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,...,total_ev,total_non_ev,density_pop,density_cs,density_ev,density_non_ev,cluster_ev,cluster_cs,cluster_non_ev,cluster_pop
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,...,446,11116,715.414905,0.290692,9.260631,230.809812,7,8,7,6
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,...,160,4014,1833.659823,0.648184,8.642459,216.817702,7,7,7,2
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,...,298,14513,1643.775383,0.565157,9.906868,482.477746,7,7,5,3
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,...,1953,37981,2035.377909,13.709375,106.670953,2074.485130,1,1,1,2
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,...,810,16366,1102.692130,6.127784,41.710125,842.750509,3,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,...,0,309,1.361529,0.000000,0.000000,2.679697,10,10,9,9
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,...,0,309,0.619576,0.000000,0.000000,0.585471,10,10,9,9
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,...,0,0,0.660622,0.000000,0.000000,0.000000,10,10,10,9
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,...,8,0,2.152604,0.000000,0.010783,0.000000,9,10,10,9


In [ ]:
for i in range(1, 11):
    print(i, df["density_pop"].loc[(df["cluster_pop"] == i)].min(), df["density_pop"].loc[(df["cluster_pop"] == i)].max())

1 2184.347988357786 19305.093842061167
2 1833.65982276567 2161.359216968873
3 1511.521311097872 1806.8078316443562
4 1233.2719509861183 1481.5315584397451
5 919.6849547242323 1210.737239214318
6 588.2494563392461 888.1174089608197
7 295.4944596543824 556.9994288856993
8 94.97489034746688 288.5508042783866
9 0.03430385344224954 93.36925129137069
10 0.0 0.0


## Grouping Annual Income

In [ ]:
th = 0.95 # threshold quantile annual income

df["cluster_ann_inc"] = np.nan
df_temp = df.copy()
temp_ann_inc = df_temp["annual_income"].loc[(df_temp["annual_income"].notnull()) &
                                            (df_temp["annual_income"] > 0)].quantile(th)
print(temp_ann_inc)
df_temp["annual_income"].loc[df_temp["annual_income"] > temp_ann_inc] = temp_ann_inc
kmeans_ann_inc = KMeans(n_clusters = 9, random_state = 42).fit(df_temp[["annual_income"]].loc[(df_temp["annual_income"].notnull())].values)
df_temp["cluster_ann_inc"].loc[df_temp["annual_income"].notnull()] = kmeans_ann_inc.predict(df_temp[["annual_income"]].loc[df_temp["annual_income"].notnull()].values)

dict_class = {}
for j in range(9):
    dict_class[j] = max(df_temp["annual_income"].loc[df_temp["cluster_ann_inc"] == j])
dict_class = dict(sorted(dict_class.items(), key=lambda item: item[1]))

df["cluster_ann_inc"].loc[(df["annual_income"].notnull())] = kmeans_ann_inc.predict(df[["annual_income"]].loc[(df["annual_income"].notnull())].values)
df["cluster_ann_inc"] = df[["cluster_ann_inc",
                            "annual_income"]].apply(lambda x: 10 if x["annual_income"] == 0 or math.isnan(x["annual_income"]) else
                                                    (9 if x["cluster_ann_inc"] == list(dict_class.keys())[0] else
                                                     (8 if x["cluster_ann_inc"] == list(dict_class.keys())[1] else
                                                      (7 if x["cluster_ann_inc"] == list(dict_class.keys())[2] else
                                                       (6 if x["cluster_ann_inc"] == list(dict_class.keys())[3] else
                                                        (5 if x["cluster_ann_inc"] == list(dict_class.keys())[4] else
                                                         (4 if x["cluster_ann_inc"] == list(dict_class.keys())[5] else
                                                          (3 if x["cluster_ann_inc"] == list(dict_class.keys())[6] else
                                                           (2 if x["cluster_ann_inc"] == list(dict_class.keys())[7] else
                                                            (1 if x["cluster_ann_inc"] == list(dict_class.keys())[8] else x))))))))), axis = 1)

joblib.dump(kmeans_ann_inc, "cluster_ann_inc_demand_internet.gz")
joblib.dump(dict_class, "dict_cluster_ann_inc_demand_internet.gz")

df

66354.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,...,total_non_ev,density_pop,density_cs,density_ev,density_non_ev,cluster_ev,cluster_cs,cluster_non_ev,cluster_pop,cluster_ann_inc
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,...,11116,715.414905,0.290692,9.260631,230.809812,7,8,7,6,5
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,...,4014,1833.659823,0.648184,8.642459,216.817702,7,7,7,2,6
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,...,14513,1643.775383,0.565157,9.906868,482.477746,7,7,5,3,6
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,...,37981,2035.377909,13.709375,106.670953,2074.485130,1,1,1,2,1
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,...,16366,1102.692130,6.127784,41.710125,842.750509,3,1,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,...,309,1.361529,0.000000,0.000000,2.679697,10,10,9,9,8
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,...,309,0.619576,0.000000,0.000000,0.585471,10,10,9,9,7
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,...,0,0.660622,0.000000,0.000000,0.000000,10,10,10,9,7
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,...,0,2.152604,0.000000,0.010783,0.000000,9,10,10,9,5


In [ ]:
for i in range(1, 11):
    print(i, df["annual_income"].loc[(df["cluster_ann_inc"] == i)].min(), df["annual_income"].loc[(df["cluster_ann_inc"] == i)].max())

1 60209 130056
2 50115 59151
3 41971 48872
4 36200 41777
5 31368 36038
6 26586 31228
7 21074 26408
8 10503 20592
9 3692 7668
10 0 0


## Grouping House Valuation

In [ ]:
th = 0.95 # threshold quantile house valuation

df["cluster_hv"] = np.nan
df_temp = df.copy()
temp_hv = df_temp["house_valuation"].loc[(df_temp["house_valuation"].notnull()) &
                                         (df_temp["house_valuation"] > 0)].quantile(th)
print(temp_hv)
df_temp["house_valuation"].loc[df_temp["house_valuation"] > temp_hv] = temp_hv
kmeans_hv = KMeans(n_clusters = 9, random_state = 42).fit(df_temp[["house_valuation"]].loc[(df_temp["house_valuation"].notnull())].values)
df_temp["cluster_hv"].loc[df_temp["house_valuation"].notnull()] = kmeans_hv.predict(df_temp[["house_valuation"]].loc[df_temp["house_valuation"].notnull()].values)

dict_class = {}
for j in range(9):
    dict_class[j] = max(df_temp["house_valuation"].loc[df_temp["cluster_hv"] == j])
dict_class = dict(sorted(dict_class.items(), key=lambda item: item[1]))

df["cluster_hv"].loc[(df["house_valuation"].notnull())] = kmeans_hv.predict(df[["house_valuation"]].loc[(df["house_valuation"].notnull())].values)
df["cluster_hv"] = df[["cluster_hv",
                       "house_valuation"]].apply(lambda x: 10 if x["house_valuation"] == 0 or math.isnan(x["house_valuation"]) else
                                                 (9 if x["cluster_hv"] == list(dict_class.keys())[0] else
                                                  (8 if x["cluster_hv"] == list(dict_class.keys())[1] else
                                                   (7 if x["cluster_hv"] == list(dict_class.keys())[2] else
                                                    (6 if x["cluster_hv"] == list(dict_class.keys())[3] else
                                                     (5 if x["cluster_hv"] == list(dict_class.keys())[4] else
                                                      (4 if x["cluster_hv"] == list(dict_class.keys())[5] else
                                                       (3 if x["cluster_hv"] == list(dict_class.keys())[6] else
                                                        (2 if x["cluster_hv"] == list(dict_class.keys())[7] else
                                                         (1 if x["cluster_hv"] == list(dict_class.keys())[8] else x))))))))), axis = 1)

joblib.dump(kmeans_hv, "cluster_hv_demand_internet.gz")
joblib.dump(dict_class, "dict_cluster_hv_demand_internet.gz")

df

735580.0000000002


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,state_id,state,county,city,zip_code,long,lat,total_population,area_land,annual_income,...,density_pop,density_cs,density_ev,density_non_ev,cluster_ev,cluster_cs,cluster_non_ev,cluster_pop,cluster_ann_inc,cluster_hv
0,WA,Washington,King,Auburn,98001,47.30919,-122.26426,34455,48.160864,35620,...,715.414905,0.290692,9.260631,230.809812,7,8,7,6,5,4
1,WA,Washington,King,Auburn,98002,47.30820,-122.21567,33947,18.513249,28275,...,1833.659823,0.648184,8.642459,216.817702,7,7,7,2,6,6
2,WA,Washington,King,Federal Way,98003,47.30596,-122.31465,49445,30.080144,30059,...,1643.775383,0.565157,9.906868,482.477746,7,7,5,3,6,5
3,WA,Washington,King,Bellevue,98004,47.61865,-122.20548,37265,18.308639,99774,...,2035.377909,13.709375,106.670953,2074.485130,1,1,1,2,1,1
4,WA,Washington,King,Bellevue,98005,47.61494,-122.16814,21414,19.419745,75452,...,1102.692130,6.127784,41.710125,842.750509,3,1,3,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,WA,Washington,Walla Walla,Wallula,99363,46.04821,-118.89575,157,115.311535,13719,...,1.361529,0.000000,0.000000,2.679697,10,10,9,9,8,8
594,WA,Washington,Adams,Washtucna,99371,46.79144,-118.30614,327,527.780392,24669,...,0.619576,0.000000,0.000000,0.585471,10,10,9,9,7,8
595,WA,Washington,Asotin,Anatone,99401,46.09041,-117.27336,376,569.160662,24091,...,0.660622,0.000000,0.000000,0.000000,10,10,10,9,7,6
596,WA,Washington,Asotin,Asotin,99402,46.18680,-117.14922,1597,741.892276,34638,...,2.152604,0.000000,0.010783,0.000000,9,10,10,9,5,6


In [ ]:
for i in range(1, 11):
    print(i, df["house_valuation"].loc[(df["cluster_hv"] == i)].min(), df["house_valuation"].loc[(df["cluster_hv"] == i)].max())

1 664200 2000001
2 542900 657800
3 443400 536100
4 360300 439400
5 292000 358400
6 224300 290800
7 155000 221100
8 64400 153300
9 56600 56600
10 0 0


# Modelling

In [ ]:
df.info()

In [ ]:
df[["density_pop",
    "annual_income",
    "house_valuation",
    "density_non_ev",
    "density_cs",
    "density_ev"]].corr()

In [ ]:
df[["cluster_pop",
    "cluster_ann_inc",
    "cluster_hv",
    "cluster_non_ev",
    "cluster_cs",
    "cluster_ev"]].corr()

In [ ]:
data = df[["cluster_pop",
           "cluster_ann_inc",
           "cluster_hv",
           "cluster_non_ev"]].values
y = df["cluster_ev"]

X_train, X_test, y_train, y_test = train_test_split(data, y, 
                                                    test_size = 0.2,
                                                    stratify = y,
                                                    random_state = 42)

ros = RandomOverSampler(random_state = 42)
X_train, y_train = ros.fit_resample(X_train, y_train)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# define two sets of inputs
inputA = tf.keras.layers.Input(shape = (X_train.shape[1]))

# the first branch operates on the first input
layer = tf.keras.layers.Dense(4)(inputA)
layer = tf.keras.layers.Dense(4)(layer)
layer = tf.keras.layers.Dense(4)(layer)
final = tf.keras.layers.Dense(1)(layer)
model = tf.keras.Model(inputs = inputA, outputs = final)

model.compile(optimizer = "RMSProp",
              loss = "mean_squared_error")

result = model.fit(X_train, y_train,
                   validation_data = (X_test, y_test),
                   epochs = 15,
                   batch_size = 32)

joblib.dump(sc, "scaler_model_demand_ev.gz")
model.save("model_demand_ev.h5")

# Evaluation

In [ ]:
df_temp = pd.DataFrame(columns = ["cluster_demand", "model_predict"])
df_temp["cluster_demand"] = y_test.values
df_temp["model_predict"] = np.round(model.predict(X_test).reshape(-1, 1)).reshape(-1)
df_temp["model_predict"].loc[df_temp["model_predict"] > 10] = 10
df_temp["model_predict"].loc[df_temp["model_predict"] < 1] = 1

print("Evaluation Without Category 10")
TP = len(df_temp.loc[(df_temp["cluster_demand"].isin([1,2,3,4,5])) &
                     (df_temp["model_predict"].isin([1,2,3,4,5]))])
FP = len(df_temp.loc[(df_temp["cluster_demand"].isin([6,7,8,9])) &
                     (df_temp["model_predict"].isin([1,2,3,4,5]))])
FN = len(df_temp.loc[(df_temp["cluster_demand"].isin([1,2,3,4,5])) &
                     (df_temp["model_predict"].isin([6,7,8,9]))])
TN = len(df_temp.loc[(df_temp["cluster_demand"].isin([6,7,8,9])) &
                     (df_temp["model_predict"].isin([6,7,8,9]))])

try:
    prec = TP/(TP+FP)
except:
    prec = 0
try:
    rec = TP/(TP+FN)
except:
    rec = 0
try:
    f1 = TP/(TP+((FP+FN)/2))
except:
    f1 = 0

print("TP =", TP)
print("FP =", FP)
print("FN =", FN)
print("TN =", TN)
print("Precision =", prec)
print("Recall =", rec)
print("F1 Score =", f1)

print("Evaluation With Category 10")
TP = len(df_temp.loc[(df_temp["cluster_demand"].isin([1,2,3,4,5])) &
                     (df_temp["model_predict"].isin([1,2,3,4,5]))])
FP = len(df_temp.loc[(df_temp["cluster_demand"].isin([6,7,8,9,10])) &
                     (df_temp["model_predict"].isin([1,2,3,4,5]))])
FN = len(df_temp.loc[(df_temp["cluster_demand"].isin([1,2,3,4,5])) &
                     (df_temp["model_predict"].isin([6,7,8,9,10]))])
TN = len(df_temp.loc[(df_temp["cluster_demand"].isin([6,7,8,9,10])) &
                     (df_temp["model_predict"].isin([6,7,8,9,10]))])

try:
    prec = TP/(TP+FP)
except:
    prec = 0
try:
    rec = TP/(TP+FN)
except:
    rec = 0
try:
    f1 = TP/(TP+((FP+FN)/2))
except:
    f1 = 0

print("TP =", TP)
print("FP =", FP)
print("FN =", FN)
print("TN =", TN)
print("Precision =", prec)
print("Recall =", rec)
print("F1 Score =", f1)

In [ ]:
df_temp = df.copy()
data = df_temp[["cluster_pop",
                "cluster_ann_inc",
                "cluster_hv",
                "cluster_non_ev"]].values
data = sc.transform(data)

df_temp["actual"] = df_temp["cluster_ev"]
df_temp["prediction"] = np.round(model.predict(data).reshape(-1, 1)).reshape(-1).astype(int)
df_temp["prediction"].loc[df_temp["prediction"] > 10] = 10
df_temp["prediction"].loc[df_temp["prediction"] < 1] = 1

print("Evaluation Without Category 10")
TP = len(df_temp.loc[(df_temp["actual"].isin([1,2,3,4,5])) &
                     (df_temp["prediction"].isin([1,2,3,4,5]))])
FP = len(df_temp.loc[(df_temp["actual"].isin([6,7,8,9])) &
                     (df_temp["prediction"].isin([1,2,3,4,5]))])
FN = len(df_temp.loc[(df_temp["actual"].isin([1,2,3,4,5])) &
                     (df_temp["prediction"].isin([6,7,8,9]))])
TN = len(df_temp.loc[(df_temp["actual"].isin([6,7,8,9])) &
                     (df_temp["prediction"].isin([6,7,8,9]))])

try:
    prec = TP/(TP+FP)
except:
    prec = 0
try:
    rec = TP/(TP+FN)
except:
    rec = 0
try:
    f1 = TP/(TP+((FP+FN)/2))
except:
    f1 = 0

print("TP =", TP)
print("FP =", FP)
print("FN =", FN)
print("TN =", TN)
print("Precision =", prec)
print("Recall =", rec)
print("F1 Score =", f1)

print("Evaluation With Category 10")
TP = len(df_temp.loc[(df_temp["actual"].isin([1,2,3,4,5])) &
                     (df_temp["prediction"].isin([1,2,3,4,5]))])
FP = len(df_temp.loc[(df_temp["actual"].isin([6,7,8,9,10])) &
                     (df_temp["prediction"].isin([1,2,3,4,5]))])
FN = len(df_temp.loc[(df_temp["actual"].isin([1,2,3,4,5])) &
                     (df_temp["prediction"].isin([6,7,8,9,10]))])
TN = len(df_temp.loc[(df_temp["actual"].isin([6,7,8,9,10])) &
                     (df_temp["prediction"].isin([6,7,8,9,10]))])

try:
    prec = TP/(TP+FP)
except:
    prec = 0
try:
    rec = TP/(TP+FN)
except:
    rec = 0
try:
    f1 = TP/(TP+((FP+FN)/2))
except:
    f1 = 0

print("TP =", TP)
print("FP =", FP)
print("FN =", FN)
print("TN =", TN)
print("Precision =", prec)
print("Recall =", rec)
print("F1 Score =", f1)

# Save Result

In [ ]:
data = df[["cluster_pop",
           "cluster_ann_inc",
           "cluster_hv",
           "cluster_non_ev"]].values
data = sc.transform(data)

df["actual"] = df["cluster_ev"]
df["prediction"] = np.round(model.predict(data).reshape(-1, 1)).reshape(-1).astype(int)
df["prediction"].loc[df["prediction"] > 10] = 10
df["prediction"].loc[df["prediction"] < 1] = 1

df = df[["zip_code",
         "long",
         "lat",
         "total_ev",
         "total_cs",
         "total_non_ev",
         "total_population",
         "annual_income",
         "house_valuation",
         "actual",
         "prediction"]]

th = 0.06 # threhold rate
df["occ_rate"] = np.nan
df["occ_rate"].loc[df["total_cs"] > 0] = df["total_cs"].loc[df["total_cs"] > 0] / df["total_ev"].loc[df["total_cs"] > 0]
df["action"] = df[["occ_rate",
                   "prediction"]].apply(lambda x: "Sales" if x["occ_rate"] > th else 
                                        ("Planning" if x["occ_rate"] <= th or math.isnan(x["occ_rate"]) else "-"), axis = 1)

df.to_csv("Data Prediction.csv", index = False, header = True)

df

In [ ]:
print("Jumlah zip code yang diperlukan action sales :", len(df.loc[df["action"] == "Sales"]))
print("Jumlah zip code yang diperlukan action plannning :", len(df.loc[df["action"] == "Planning"]))

In [ ]:
print("Jumlah zip code yang diperlukan action sales :", len(df.loc[(df["action"] == "Sales") &
                                                            (df["prediction"] <= 5)]))
print("Jumlah zip code yang diperlukan action plannning :", len(df.loc[(df["action"] == "Planning") &
                                                                (df["prediction"] <= 5)]))